<a href="https://colab.research.google.com/github/miguelmccormickudg/Gravitational_housing_model/blob/main/modelo_parte_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# -------------------------------
# Paso 1: Cargar los datos originales desde el archivo Excel
# -------------------------------
file_path = "/content/para red social 998 registros.xlsx"
df = pd.read_excel(file_path)

# Limpiar nombres de columnas (eliminar espacios, poner en minúsculas)
df.columns = df.columns.str.strip().str.lower()

# -------------------------------
# Paso 2: Calcular M_host
# -------------------------------

# Asegurar tipos numéricos
for col in ['host_rating', 'host_count', 'host_years', 'reviewscount']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Definir función para limpieza de ubicación
def clean_location(loc):
    if isinstance(loc, str):
        loc = loc.lower()
        if 'ajijic' in loc or 'chapala' in loc:
            return 'Ajijic/Chapala'
        elif 'guadalajara' in loc:
            return 'Guadalajara'
        elif 'zapopan' in loc:
            return 'Zapopan'
        else:
            return 'Otro'
    return 'Desconocido'

# Contar cuántas veces aparece cada host
df['host_count_total'] = df.groupby('host_name')['host_name'].transform('count')

# Superhost como valor binario
# Change 'Superhost' to 'superhost' because column names were lowercased
df['Superhost_value'] = df['superhost'].apply(lambda x: 1 if x == True else 0)

# Contar cohosts por fila
df['cohosts_count'] = df[['cohosts_1', 'cohosts_2', 'cohosts_3']].notnull().sum(axis=1)

# Asignar puntuación de ubicación
df['location_score'] = df['host_location'].apply(clean_location).map({
    'Ajijic/Chapala': 3,
    'Guadalajara': 2,
    'Zapopan': 2,
    'Otro': 1
})

# Calcular M_host
df['M_host'] = (
    df['host_count_total'] +
    df['Superhost_value'] +
    df['cohosts_count'] +
    df['location_score'] +
    df['host_rating'] +
    df['host_count'] +
    df['host_years'] +
    df['reviewscount']
)

# Guardar resultado de M_host
output_host = "/content/M_host_resultado.csv"
df.to_csv(output_host, index=False)

print(f"Archivo guardado: {output_host}")

# -------------------------------
# Paso 3: Calcular M_property a partir de M_host y datos de propiedades
# -------------------------------

# Unir con datos de propiedades (en este caso es el mismo DataFrame)
df_props = df.copy()

# Definir función para mapear valores binarios
def map_binary(value, true_val=2, false_val=0):
    return true_val if value == True else false_val

# room_Category: 2=private_room, 1=entire_home
df_props['room_category_value'] = df_props['room_category'].apply(
    lambda x: 2 if x == 'private_room' else 1 if x == 'entire_home' else 0
)

# petsAllowed y canInstantBook
df_props['petsAllowed_value'] = df_props['petsallowed'].apply(map_binary)
df_props['canInstantBook_value'] = df_props['caninstantbook'].apply(map_binary)

# Contar amenities
amenity_cols = [f'amenities/{i}/title' for i in range(10)]
df_props['amenities_count'] = df_props[amenity_cols].notnull().sum(axis=1)

# Calcular M_property
df_props['M_property'] = (
    df_props['M_host'] +
    df_props['room_category_value'] +
    df_props['listing_rating'] + # Ensure this column name is also lowercased if necessary
    df_props['reviews'] + # Ensure this column name is also lowercased if necessary
    df_props['maxguest'] + # Ensure this column name is also lowercased if necessary
    df_props['petsAllowed_value'] +
    df_props['canInstantBook_value'] +
    df_props['amenities_count']
)

# Guardar resultado de M_property
output_property = "/content/M_property_resultado.csv"
df_props.to_csv(output_property, index=False)

print(f"Archivo guardado: {output_property}")

# -------------------------------
# Paso 4: Calcular masa total del sistema para **Ajijic**
# -------------------------------

# Filtrar hosts que están en Ajijic (basado en host_location)
ajijic_hosts = df[df['host_location'].str.contains('ajijic|chapala', case=False, na=False)]['host_name']

# Filtrar propiedades de hosts en Ajijic
df_props_ajijic = df_props[df_props['host_name'].isin(ajijic_hosts)]

# Sumar masas
total_m_host = df[df['host_name'].isin(ajijic_hosts)]['M_host'].sum()
total_m_property = df_props_ajijic['M_property'].sum()

# Peso fijo de actores globales
airbnb_weight = 7700000 + 5000000  # 12,700,000
gov_weight = 12700 + 11500 + 1392226  # 1,416,426

# Calcular masa total de Ajijic
masa_total_ajijic = (
    total_m_host +
    total_m_property +
    airbnb_weight +
    gov_weight
)

# Guardar resultado final
final_df = pd.DataFrame({
    'localidad': ['Ajijic'],
    'masa_total': [masa_total_ajijic]
})
final_output = "/content/masa_total_ajijic.csv"
final_df.to_csv(final_output, index=False)

print(f"\n✅ Masa total de Ajijic: {masa_total_ajijic}")
print(f"Archivo exportado: {final_output}")

Archivo guardado: /content/M_host_resultado.csv
Archivo guardado: /content/M_property_resultado.csv

✅ Masa total de Ajijic: 14135242.65
Archivo exportado: /content/masa_total_ajijic.csv


In [ ]:
import pandas as pd
import numpy as np

# -------------------------------
# Paso 1: Cargar los datos desde el archivo Excel
# -------------------------------
file_path = "/content/para red social 998 registros.xlsx"
df = pd.read_excel(file_path)

# Limpiar nombres de columnas
df.columns = df.columns.str.strip().str.lower()

# -------------------------------
# Paso 2: Identificar columnas de coordenadas
# -------------------------------

lat_col = None
lon_col = None

for col in df.columns:
    if "latitude" in col.lower():
        lat_col = col
    if "longitude" in col.lower():
        lon_col = col

if not lat_col or not lon_col:
    raise ValueError("No se encontraron columnas de coordenadas (latitud y longitud)")

# Convertir a numérico y limpiar valores nulos
df[lat_col] = pd.to_numeric(df[lat_col], errors='coerce')
df[lon_col] = pd.to_numeric(df[lon_col], errors='coerce')

# Filtrar filas con coordenadas válidas
df_coords = df.dropna(subset=[lat_col, lon_col]).copy()
coords = df_coords[[lat_col, lon_col]].values

print(f"\nSe encontraron {len(coords)} registros con coordenadas válidas.")

# -------------------------------
# Paso 3: Definir función de distancia Haversine
# -------------------------------
def haversine_distance(lat1, lon1, lat2, lon2, r=6371):
    # Convertir grados a radianes
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # Diferencias
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Fórmula de Haversine
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))

    return r * c  # Distancia en km

# -------------------------------
# Paso 4: Calcular matriz de distancias entre todos los puntos
# -------------------------------
n = len(coords)
distance_matrix = np.zeros((n, n))

for i in range(n):
    for j in range(i + 1, n):
        lat1, lon1 = coords[i]
        lat2, lon2 = coords[j]
        distance_matrix[i, j] = haversine_distance(lat1, lon1, lat2, lon2)
        distance_matrix[j, i] = distance_matrix[i, j]  # Simétrico

# Guardar resultado como CSV
output_path = "/content/distance_matrix.csv"
pd.DataFrame(distance_matrix, index=df_coords.index, columns=df_coords.index).to_csv(output_path)

print(f"\n✅ Matriz de distancias calculada y guardada en: {output_path}")


Se encontraron 320 registros con coordenadas válidas.

✅ Matriz de distancias calculada y guardada en: /content/distance_matrix.csv


In [ ]:
import pandas as pd

# Cargar los datos
file_path = "/content/para red social 998 registros.xlsx"
df = pd.read_excel(file_path)

# Limpiar nombres de columnas
df.columns = df.columns.str.strip().str.lower()

# Identificar columnas relevantes
host_count_col = None
reviews_count_col = None
host_years_col = None

for col in df.columns:
    if 'host_count' in col:
        host_count_col = col
    elif 'reviewscount' in col:
        reviews_count_col = col
    elif 'host_years' in col:
        host_years_col = col

# Asegurar que tengamos las columnas necesarias
if not all([host_count_col, reviews_count_col, host_years_col]):
    raise ValueError("Faltan columnas necesarias para calcular E_i")

# Convertir a numérico
df[host_count_col] = pd.to_numeric(df[host_count_col], errors='coerce')
df[reviews_count_col] = pd.to_numeric(df[reviews_count_col], errors='coerce')
df[host_years_col] = pd.to_numeric(df[host_years_col], errors='coerce')

# Evitar división por cero
df[host_years_col] = df[host_years_col].apply(lambda x: x if x != 0 else 1)

# Calcular E_i
df['E_i'] = (df[host_count_col] + df[reviews_count_col]) / df[host_years_col]

# Mostrar resultados
print("\nTop hosts por E_i:")
print(df[['host_name', 'E_i']].sort_values(by='E_i', ascending=False).head(10))

# Exportar resultado final
output_path = "/content/E_i_resultado.csv"
df.to_csv(output_path, index=False)

print(f"\n✅ Archivo guardado en: {output_path}")


Top hosts por E_i:
                                             host_name     E_i
211                        Studio with Salt Water Pool  1041.0
210  Ajijic Retreat Studio with Outdoor Salt Water ...   982.0
209     Poolside Paradise Suite in the heart of Ajijic   946.0
205  Tranquil Studio Suite with Outdoor Salt Water ...   942.0
206      Serene Oasis Studio - Outdoor Salt Water Pool   915.0
207      Serene Oasis Studio - Outdoor Salt Water Pool   893.0
208    Soothing Studio Suite - Outdoor Salt Water Pool   890.0
10             King-size room in Chapala Lake, Ajijic!   657.0
8         Two bedroom apartment - two floors near lake   580.0
9                      Apartment with a king-size bed!   560.0

✅ Archivo guardado en: /content/E_i_resultado.csv


In [ ]:
import pandas as pd
import numpy as np

# Cargar datos
file_path = "/content/para red social 998 registros.xlsx"
df = pd.read_excel(file_path)

# Limpiar nombres de columnas
df.columns = df.columns.str.strip().str.lower()

# -------------------------------
# Paso 1: Elegir variable para calcular entropía
# -------------------------------

# Opciones: 'host_location', 'room_category', 'superhost'
variable = 'host_location'  # Puedes cambiar por 'room_category' o 'superhost'

# Filtrar valores no vacíos
df_valid = df[df[variable].notnull()]

# Contar frecuencias
freq = df_valid[variable].value_counts()

# Convertir a probabilidades
probabilities = freq / freq.sum()

# Calcular entropía
entropy = -np.sum(probabilities * np.log2(probabilities))

# Mostrar resultado
print("Distribución de frecuencias:")
print(freq)
print("\nProbabilidades:")
print(probabilities)
print(f"\n📊 Entropía de Shannon (H): {entropy:.4f} bits")

# Interpretación
if entropy < 0.5:
    print("🔹 Alta concentración: El sistema está muy dominado por un grupo o zona")
elif entropy < 1.5:
    print("🔸 Moderada dispersión: Hay cierta diversidad pero con algunos dominantes")
else:
    print("🔺 Alta dispersión: Los elementos están ampliamente distribuidos")

Distribución de frecuencias:
host_location
Lives in Guadalajara, Mexico               26
Lives in Ajijic, Mexico                    23
Lives in Zapopan, Mexico                   15
Lives in Chapala, Mexico                    9
Lives in San Juan Cosalá, Mexico            6
Lives in Aguascalientes, Mexico             6
Lives in San Antonio Tlayacapan, Mexico     4
Lives in Jalisco, Mexico                    3
Lives in Mexico                             2
Lives in Chicago, Illinois                  2
Lives in Atlanta, Georgia                   2
Lives in Tlaquepaque, Mexico                2
Lives in Wayne, Pennsylvania                1
Lives in Vancouver, Canada                  1
Lives in Kennesaw, Georgia                  1
Lives in La Tijera, Mexico                  1
Lives in Spring, Texas                      1
Lives in Mexico City, Mexico                1
Lives in Iguala, Mexico                     1
Lives in West Vancouver, Canada             1
Lives in Jocotepec, Mexico           

In [ ]:
import pandas as pd
import numpy as np

# Cargar los datos desde Excel
file_path = "/content/para red social 998 registros.xlsx"
df = pd.read_excel(file_path)

# Limpiar nombres de columnas
df.columns = df.columns.str.strip().str.lower()

# -------------------------------
# Paso 1: Identificar variables relevantes
# -------------------------------

# Detectar automáticamente las columnas clave
host_col = next((col for col in df.columns if 'host_name' in col), None)
room_category_col = next((col for col in df.columns if 'room_category' in col), None)
location_col = next((col for col in df.columns if 'host_location' in col), None)
amenity_cols = [col for col in df.columns if 'amenities/' in col]

# Validar que al menos haya alguna columna para calcular entropía
if not any([host_col, room_category_col, location_col, len(amenity_cols) > 0]):
    raise ValueError("No se encontraron columnas suficientes para calcular entropía")

# -------------------------------
# Paso 2: Función auxiliar para calcular entropía de una variable
# -------------------------------
def calculate_entropy(series):
    freq = series.value_counts(normalize=True) + 1e-10  # Evitar log(0)
    entropy = -np.sum(freq * np.log2(freq))
    return entropy

# -------------------------------
# Paso 3: Calcular entropía por fila (propiedad)
# -------------------------------

# Definir función para calcular entropía por fila
def get_row_entropy(row):
    values = []

    # Añadir host_name si existe
    if host_col:
        values.append(str(row[host_col]))

    # Añadir room_Category si existe
    if room_category_col:
        values.append(str(row[room_category_col]))

    # Añadir host_location si existe
    if location_col:
        values.append(str(row[location_col]))

    # Añadir amenities no vacíos
    for col in amenity_cols:
        val = str(row[col])
        if val and val != 'nan' and val != 'None':
            values.append(val)

    # Si no hay valores, entropía = 0
    if not values:
        return 0.0

    # Calcular entropía relativa
    freq = pd.Series(values).value_counts(normalize=True) + 1e-10
    row_entropy = -np.sum(freq * np.log2(freq))

    return row_entropy

# Aplicar la función fila por fila
df['entropy_weight'] = df.apply(get_row_entropy, axis=1)

# -------------------------------
# Paso 4: Mostrar resultados y exportar
# -------------------------------

print("\nTop 10 propiedades con mayor peso de entropía:")
print(df[['host_name', 'entropy_weight']].sort_values(by='entropy_weight', ascending=False).head(10))

# Exportar resultado final
output_path = "/content/listings_with_entropy.csv"
df.to_csv(output_path, index=False)

print(f"\n✅ Archivo guardado en: {output_path}")


Top 10 propiedades con mayor peso de entropía:
                                          host_name  entropy_weight
303                      Nice apartment with a pool         3.70044
302              Complete apartment Flor de paraíso         3.70044
301                        Ajijic Centro Sandy loft         3.70044
300               Best place and location in Ajijic         3.70044
299                   Casa Victoria downtown Ajijic         3.70044
298  Chapala Central.. all new beautiful convenient         3.70044
297                Exclusive house in "El Tepetate"         3.70044
296                              Vista Lago Chapala         3.70044
295     Cozy apartment with pool and terrace. Seuia         3.70044
294       Cozy apartment overlooking the Seuia pool         3.70044

✅ Archivo guardado en: /content/listings_with_entropy.csv


In [ ]:
# -------------------------------
# Paso 0: Instalar dependencias
# -------------------------------
!pip install -q simplekml

# -------------------------------
# Paso 1: Cargar datos desde Excel
# -------------------------------
import pandas as pd
import numpy as np
import simplekml

file_path = "/content/para red social 998 registros.xlsx"

# Cargar la hoja que contiene los listados (ajusta el nombre si es necesario)
df = pd.read_excel(file_path)

# Limpiar nombres de columnas
df.columns = df.columns.str.strip().str.lower()

# -------------------------------
# Paso 2: Calcular ∑M_k (masa total del sistema)
# -------------------------------

# Usamos M_property calculado previamente o lo simulamos si no está disponible
if 'M_property' in df.columns:
    SUM_MK = df['M_property'].sum()
else:
    # Simular masa aproximada si no existe
    df['M_property'] = 100 + np.random.rand(len(df)) * 50
    SUM_MK = df['M_property'].sum()

print(f"\n✅ Masa total del sistema (∑M_k): {SUM_MK}")

# -------------------------------
# Paso 3: Calcular D_ij usando Haversine entre todos los puntos
# -------------------------------

lat_col = None
lon_col = None
for col in df.columns:
    if 'latitude' in col:
        lat_col = col
    if 'longitude' in col:
        lon_col = col

if lat_col and lon_col:
    from math import radians, sin, cos, sqrt, atan2

    def haversine(lat1, lon1, lat2, lon2, r=6371):
        lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
        c = 2 * atan2(sqrt(a), sqrt(1-a))
        return r * c

    coords = df[[lat_col, lon_col]].apply(pd.to_numeric, errors='coerce').values
    n = len(coords)
    distance_matrix = np.zeros((n, n))

    for i in range(n):
        for j in range(i+1, n):
            lat1, lon1 = coords[i]
            lat2, lon2 = coords[j]
            distance_matrix[i, j] = haversine(lat1, lon1, lat2, lon2)
            distance_matrix[j, i] = distance_matrix[i, j]

    average_distances = np.nanmean(np.where(distance_matrix == 0, np.nan, distance_matrix), axis=1)
    df['D_i'] = average_distances
else:
    df['D_i'] = 1  # Valor por defecto si no hay coordenadas

# -------------------------------
# Paso 4: Cargar E_i (Estabilidad) si ya existe o simularla
# -------------------------------

if 'E_i' not in df.columns:
    df['host_count'] = pd.to_numeric(df['host_count'], errors='coerce').fillna(1)
    df['host_years'] = pd.to_numeric(df['host_years'], errors='coerce').fillna(1)
    df['reviewscount'] = pd.to_numeric(df['reviewscount'], errors='coerce').fillna(0)
    df['E_i'] = (df['host_count'] + df['reviewscount']) / df['host_years']

# -------------------------------
# Paso 5: Usar S_i fijo (entropía Shannon)
# -------------------------------

S_i = 3.5887  # Valor proporcionado

df['S_i'] = S_i

# -------------------------------
# Paso 6: Aplicar la fórmula final F_ij
# -------------------------------

beta = 2  # Puedes ajustarlo según el contexto

df['F_ij'] = (SUM_MK / (df['D_i'] ** beta)) * df['E_i'] * df['S_i']

# -------------------------------
# Paso 7: Exportar CSV con todas las columnas + F_ij
# -------------------------------

output_csv = "/content/F_ij_resultado.csv"
df.to_csv(output_csv, index=False)
print(f"\n✅ Archivo CSV guardado en: {output_csv}")

# -------------------------------
# Paso 8: Exportar KML con coordenadas y F_ij
# -------------------------------

kml = simplekml.Kml()

host_col = next((col for col in df.columns if 'host_name' in col), None)
lat_col = next((col for col in df.columns if 'latitude' in col), None)
lon_col = next((col for col in df.columns if 'longitude' in col), None)

if host_col and lat_col and lon_col:
    for _, row in df.iterrows():
        lat = row[lat_col]
        lon = row[lon_col]
        name = str(row[host_col])[:30]
        desc = f"F_ij: {row['F_ij']:.2f}\n" + "\n".join([f"{col}: {row[col]}" for col in df.columns if pd.notna(row[col])][:10])

        if pd.notna(lat) and pd.notna(lon):
            point = kml.newpoint(name=name, description=desc, coords=[(lon, lat)])
            point.style.iconstyle.color = 'ff0000ff'  # Color rojo

    output_kml = "/content/F_ij_resultado.kml"
    kml.save(output_kml)
    print(f"✅ Archivo KML guardado en: {output_kml}")
else:
    print("⚠️ No se encontraron columnas necesarias para KML")


✅ Masa total del sistema (∑M_k): 39539.477678558265

✅ Archivo CSV guardado en: /content/F_ij_resultado.csv
✅ Archivo KML guardado en: /content/F_ij_resultado.kml
